# Numerical Simulation of Ballistic Performance of Honeycomb Sandwich Panels

### Objective:
Setup a parametric model in PyMAPDL for a honeycomb sandwich panel. Use it for running a Ballistic impact simulation

The learning objectives of this demo are:
* Setup and solve a parametric model using PyMAPDL
* Interactive plotting of CAD, mesh, and results in Pythonic interface.

### Model Details

#### Model parameters:
* Cell size
* Cell wall thickness
* Node length
* Facesheet thickness

## Step 1 - define all parameters

In [9]:
## Main parameters
cell_size = 0.006
cell_wall_thickness = 0.002
node_length = 0.004
facesheet_thickness = 0.005

In [10]:
# Fixed constants
strucutre_length = 0.1
strucutre_breadth = 0.1

In [11]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Additional calculation 
L = cell_size/ np.sqrt(3)
offset_x = cell_size + cell_wall_thickness
offset_y = 1.5*L + cell_wall_thickness*(np.sin(np.pi/3))
Nx = int(strucutre_length / offset_x) + 1
Ny = int(strucutre_breadth / offset_y) + 1
print("L = ", L)
print("offset_x = ", offset_x)
print("offset_y = ", offset_y)
print("Nx = ", Nx)
print("Ny = ", Ny)

L =  0.0034641016151377548
offset_x =  0.008
offset_y =  0.0069282032302755096
Nx =  13
Ny =  15


# Step 2 - launch MAPDL and create geometry

In [12]:
from ansys.mapdl.core import launch_mapdl

# start mapdl
mapdl = launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [13]:
## reset mapdl
mapdl.clear()
mapdl.prep7()

## Create geometry
mapdl.block(0, strucutre_length, 0, strucutre_breadth, 0, node_length)
for i in range(0,Nx):
    for j in range(0,Ny):
        if j%2 == 0:
            mapdl.rpr4(6, i*offset_x , j*offset_y, L, 90, node_length)
        else:
            mapdl.rpr4(6, i*offset_x + offset_x/2 , j*offset_y, L, 90, node_length)
mapdl.vsbv(1,'all')
mapdl.cm("CORE", "VOLU")
mapdl.block(0, strucutre_length, 0, strucutre_breadth, node_length, node_length + facesheet_thickness)  # Top facesheet
mapdl.block(0, strucutre_length, 0, strucutre_breadth, -1*facesheet_thickness, 0)                       # Bottom facesheet

mapdl.vplot('all')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 3 - define material properties, mesh attributes and generate mesh

In [14]:
## Define material properties
mapdl.mp("EX", 1, 2e5)  # Youngs modulus
mapdl.mp("PRXY", 1, 0.3367)  # Poissons ratio

## Define element attributes
mapdl.et(1, "SOLID185") # 3D 8-Node Layered Solid

## Define mesh controls

mapdl.lesize("ALL", 0.001, layer1=1)
# mapdl.smrtsize(1)

# Mesh facesheets
mapdl.mshape(0, "3D")    # mesh the volume with 3D Hexahedral elements
mapdl.mshkey(1)          # mapped mesh
mapdl.vmesh('2')
mapdl.mshape(0, "3D")    # mesh the volume with 3D Hexahedral elements
mapdl.mshkey(1)          # mapped mesh
mapdl.vmesh('1')

# Mesh honeycomb core
mapdl.mshape(1, "3D")    # mesh the volume with 3D Tetrahedral elements
mapdl.mshkey(0)          # free mesh
mapdl.vmesh('CORE')

## Plot the mesh
mapdl.eplot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 4 - apply loads and boundary conditions

In [15]:
### Make the lateral sides of the panel as fixed support i.e. set displacement of nodes

# Select all nodes at the x = 0 end of the sandwich panel
mapdl.nsel("S", "LOC", "X", 0)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("ALL", "UX")
mapdl.d("ALL", "UY")
mapdl.d("ALL", "UZ")

# Select all nodes at the x = structure_length end of the sandwich panel
mapdl.nsel("S", "LOC", "X", strucutre_length)
# Remove all degrees of freedom for all nodes in the selection
#mapdl.d("all", "all")
mapdl.d("ALL", "UX")
mapdl.d("ALL", "UY")
mapdl.d("ALL", "UZ")

# Select all nodes at the y = 0 end of the sandwich panel
mapdl.nsel("S", "LOC", "Y", 0)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("ALL", "UX")
mapdl.d("ALL", "UY")
mapdl.d("ALL", "UZ")

# Select all nodes at the y = structure_breadth end of the sandwich panel
mapdl.nsel("S", "LOC", "Y", strucutre_breadth)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("ALL", "UX")
mapdl.d("ALL", "UY")
mapdl.d("ALL", "UZ")

SPECIFIED CONSTRAINT UZ   FOR SELECTED NODES            1 TO      173366 BY           1
 REAL=  0.00000000       IMAG=  0.00000000

In [16]:
mapdl.exit()